In [1]:
from jupyter_cadquery import set_defaults, set_sidecar
from jupyter_cadquery.cadquery import (PartGroup, Part, show as show_object)

set_defaults(theme="dark")
set_sidecar("CadQuery", init=True)
import cadquery as cq
print(cq.__version__)

Overwriting auto display for cadquery Workplane and Shape


/home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


2.1


In [17]:
from math import sin, cos, pi
SPRING_THICK = 5
SPRING_DIA = 30

L = 60
W = 30
H = 40

P = H/4
T = H/6-0.7

def spring(dia=W, height=H, pitch=P, thick=T):
    helix = cq.Workplane('XY').parametricCurve(
        lambda t: (dia/2 * cos(t*2*pi), dia/2 * sin(t* 2*pi), pitch * t),
        start = 0, stop = (height-thick)/pitch)
    return cq.Workplane('XZ').center(dia/2, 0).rect(thick, thick).sweep(helix, isFrenet=True).translate((0,0,thick/2))

# spring()
def grip(w, d, h, thick):
    coil_strut = (
        cq.Workplane().center(-w, 0).circle((w+thick)/2).circle((w-thick)/2).extrude(thick)
        .union(cq.Workplane("YZ").center(0, thick/2).rect(thick, thick).extrude(-w/2))
        .translate((-w, 0, -h/2))        
    )
    return (cq.Workplane().slot2D(d+thick, w+thick).extrude(h/2, both=True).cut(
                cq.Workplane().slot2D(d, w).extrude(h/2, both=True))
                .faces(">Z").workplane()
                .transformed(rotate=cq.Vector(90, 0, 0))
                .split(keepTop=True)
                .translate((0, -1, 0))
                .union(coil_strut)
                .cut(cq.Workplane("YZ").center(0, h/2).rect(1.2*thick, 2*thick).extrude(-2*d))
           )
    
lgrip = grip(W, L, H, T)
rgrip = grip(W, L, H, T).rotate((0,0,0),(1,0,0),180)
part = (
    spring() # .translate((-L,0,-H/2))
    .union(lgrip).union(rgrip)
)
part

Done, using side car 'Cadquery'


# Spincaster Button Cap

Enlarged caps for tactile switches mounted to the DIY spincaster. 

Ream with a 3.2mm drill bit before attaching

Engage silly mode for more butt.

In [ ]:
show_object(button)

In [58]:
from pathlib import Path; downloads = str(Path.home() / "Downloads")
from mkr3d.commands import post
for k, p in parts.items():
    path = f"{downloads}/{k}"
    cq.exporters.export(p, path)
    print("exported", path)
    post(path, slice=True, queue=True, publish=True)
print("done")

exported /home/semartin/Downloads/shaft_jig_square.stl
{'exec_dir': '/home/semartin/Documents/devenv/mkr3d/grid-apps'}
/home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/mkr3d/kirimoto.py
node /home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/mkr3d/kiri-cli.js /home/semartin/Documents/devenv/mkr3d/grid-apps /home/semartin/.config/mkr3d/km_slicer.json /home/semartin/.config/mkr3d/km_device.json /home/semartin/Downloads/shaft_jig_square.stl /home/semartin/Downloads/shaft_jig_square.gcode
settings_path /home/semartin/.config/mkr3d/km_slicer.json
device_path /home/semartin/.config/mkr3d/km_device.json
{
  msg: 'failed to load',
  dir: '/home/semartin/Documents/devenv/mkr3d/grid-apps',
  file: '/home/semartin/Documents/devenv/mkr3d/grid-apps/src/geo/debug.js'
}
{ fake_worker: '/code/worker.js?3.0.5' }
kiri | init work | 3.0.5
[
  'driver',     'loader',    'license',
  'version',    'copyright', 'conf',
  'work',       'client',    'newEngine',
  'api',        '